<a href="https://colab.research.google.com/github/yashrajOjha/multilingual-ner/blob/main/Testing_XLM_Roberta_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
!unzip '/content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/trained-ner-xlm-hindi-ai4bharat/trained_ner_classifier_hin.zip'

Archive:  /content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/trained-ner-xlm-hindi-ai4bharat/trained_ner_classifier_hin.zip
   creating: content/trained_ner_classifier_hin/
  inflating: content/trained_ner_classifier_hin/config.json  
  inflating: content/trained_ner_classifier_hin/pytorch_model.bin  


In [3]:
!unzip '/content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/trained_ner_classifier_kan.zip'

Archive:  /content/gdrive/MyDrive/cflit-hiner-xlm-roberta-weights/trained_ner_classifier_kan.zip
   creating: content/trained_ner_classifier_kan/
  inflating: content/trained_ner_classifier_kan/config.json  
  inflating: content/trained_ner_classifier_kan/pytorch_model.bin  


In [ ]:
!pip install numpy seqeval pandas matplotlib scikit-learn transformers datasets

In [5]:
import torch

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is not enabled.


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('ai4bharat/naamapadam','hi',split='train[:9%]')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


In [ ]:
tags = train_dataset.features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
index2tag

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

In [6]:
num_labels = 7
print(num_labels)
# print(tags.num_classes)

tag2index = {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6}

index2tag = {0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

7


In [7]:
from transformers import AutoTokenizer
xlmr_tokenizer = AutoTokenizer.from_pretrained("cfilt/HiNER-original-xlm-roberta-large")
print("The maximum length for the inputs is {}".format(xlmr_tokenizer.model_max_length))

The maximum length for the inputs is 512


In [13]:
import torch
from transformers import AutoConfig
from transformers.models.roberta.modeling_roberta import RobertaForTokenClassification

xlmr_config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
    num_labels=num_labels,
    id2label=index2tag,
    label2id=tag2index
)
model_kan = (RobertaForTokenClassification
         .from_pretrained("/content/content/trained_ner_classifier_kan", config=xlmr_config)
         .to(device))

In [9]:
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
import pandas as pd

sample_encoding = xlmr_tokenizer([
   "हावड़ा जंक्शन रेलवे स्टेशन",
   "पंडित नेहरू महान देशभक्त थे",
   "स्टूडियो से बाहर",
   "मोदीजी एक अच्छे नेता हैं",
   "यशराज आज गोरखपुर जाएगा",
   "उत्तर प्रदेश के परिवार से मिलने गए योगी जी, परिवहन निगम से कराई बात।",
   "अमेरिका के राष्ट्रपति बराक ओबामा मिलने आए परिवार से भारत के दिल्ली में",
   "ಭಾರತ ವಿಶ್ವ ಗುರುವಾಗಲು ಕರ್ನಾಟಕದಿಂದ ದೊಡ್ಡ ಕೊಡುಗೆ: ಸಿಎಂ ಬಸವರಾಜ ಬೊಮ್ಮಾಯಿ",
   "ಯಶರಾಜ್ ನಾಳೆ ಗೋರಖ್‌ಪುರಕ್ಕೆ ಹೋಗುತ್ತಿದ್ದಾರೆ, ಅದು ಭಾರತದ ನಗರ"
  # "सेक्टर 55/56 क एसएचओ अरविंद कुमार ने बताया कि इस मामले"
], truncation=True, max_length=512)
sample_dataset = Dataset.from_dict(sample_encoding)
sample_dataset = sample_dataset.with_format("torch")

sample_dataloader = DataLoader(sample_dataset, batch_size=1)
tokens = []
labels = []
for batch in sample_dataloader:
    # predict
    with torch.no_grad():
        output = model_hin(batch["input_ids"].to(device), batch["attention_mask"].to(device))
    predicted_label_id = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # create output
    tokens.append(xlmr_tokenizer.convert_ids_to_tokens(batch["input_ids"][0]))
    labels.append([index2tag[i] for i in predicted_label_id[0]])

In [10]:
pd.DataFrame([tokens[0], labels[0]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁हा,व,ड़ा,▁ज,ं,क्शन,▁रेलवे,▁स्टेशन,</s>
Tags,O,B-LOC,B-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,O


In [11]:
pd.DataFrame([tokens[1], labels[1]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁पंडित,▁ने,हरू,▁महान,▁देश,भक्त,▁थे,</s>
Tags,O,B-PER,I-PER,I-PER,O,O,O,O,O


In [12]:
pd.DataFrame([tokens[5], labels[5]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Tokens,<s>,▁उत्तर,▁प्रदेश,▁के,▁परिवार,▁से,▁मिलने,▁गए,▁योगी,▁जी,",",▁परिवहन,▁निगम,▁से,▁कर,ाई,▁बात,।,</s>
Tags,O,B-LOC,I-LOC,O,O,O,O,O,B-PER,O,O,B-ORG,I-ORG,O,O,O,O,O,O


In [14]:
pd.DataFrame([tokens[7], labels[7]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁ಭಾರತ,▁ವಿಶ್ವ,▁ಗುರು,ವಾಗ,ಲು,▁ಕರ್ನಾಟಕ,ದಿಂದ,▁ದೊಡ್ಡ,▁ಕೊಡುಗೆ,:,▁ಸಿಎಂ,▁ಬ,ಸ,ವ,ರಾಜ,▁ಬೊ,ಮ್ಮ,ಾಯಿ,</s>
Tags,O,B-LOC,O,O,O,O,B-LOC,O,O,O,O,O,B-PER,B-PER,I-PER,I-PER,I-PER,I-PER,I-PER,O


In [15]:
pd.DataFrame([tokens[8], labels[8]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Tokens,<s>,▁,ಯ,ಶ,ರಾಜ್,▁ನಾಳೆ,▁ಗೋ,ರ,ಖ್,▁,ಪುರ,ಕ್ಕೆ,▁ಹೋಗ,ುತ್ತಿದ್ದಾರೆ,",",▁ಅದು,▁ಭಾರತದ,▁ನಗರ,</s>
Tags,O,B-PER,B-PER,B-PER,I-PER,O,B-LOC,B-LOC,I-LOC,I-LOC,I-LOC,O,O,O,O,O,O,O,O


# Evaluation on Test Set

In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('ai4bharat/naamapadam','hi',split='test[:9%]')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


In [ ]:
test_dataset = train_dataset
test_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 39
})

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_ds = test_dataset.map(
    tokenize_and_align_labels,
    remove_columns=["tokens", "ner_tags"],
    batched=True,
    batch_size=128)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 39
})

In [ ]:
tokens = []
labels = []
index2tag.setdefault(-100,'Pad')
for ind,val in enumerate(tokenized_ds):
  token = xlmr_tokenizer.convert_ids_to_tokens(val["input_ids"])
  i = token.index('</s>')
  token = token[:i+1]
  label = [index2tag[i] for i in val["labels"]]
  label = label[:i+1]
  label = ['O' if x=='Pad' else x for x in label]
  tokens.append(token)
  labels.append(label)

In [ ]:
labels[1]

['O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
finallist_str = []
for ind,val in enumerate(test_dataset['tokens']):
  finallist_str.append(" ".join(val))

In [ ]:
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
import pandas as pd

sample_encoding = xlmr_tokenizer(
     finallist_str,
    truncation=True, max_length=512)
sample_dataset = Dataset.from_dict(sample_encoding)
sample_dataset = sample_dataset.with_format("torch")

sample_dataloader = DataLoader(sample_dataset, batch_size=1)
tokens_pred = []
labels_pred = []
for batch in sample_dataloader:
    # predict
    with torch.no_grad():
        output = model(batch["input_ids"].to(device), batch["attention_mask"].to(device))
    predicted_label_id = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # create output
    tokens_pred.append(xlmr_tokenizer.convert_ids_to_tokens(batch["input_ids"][0]))
    labels_pred.append([index2tag[i] for i in predicted_label_id[0]])

In [ ]:
pd.DataFrame([tokens_pred[3], labels_pred[3],labels[3]], index=["Tokens", "Pred_Tags","Actual Tags"]).T

,Tokens,Pred_Tags,Actual Tags
0,<s>,O,O
1,▁सोशल,O,O
2,▁मीडिया,O,O
3,▁पर,O,O
4,▁सबसे,O,O
5,▁पहले,O,O
6,▁इस,O,O
7,▁वीडियो,O,O
8,▁को,O,O
9,▁इंडियन,B-ORG,B-ORG


In [ ]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [ ]:
f1_score(labels, labels_pred)

0.5114155251141552

In [ ]:
accuracy_score(labels, labels_pred)

0.8982229402261712

In [ ]:
print(classification_report(labels, labels_pred))

              precision    recall  f1-score   support

         LOC       0.57      0.69      0.62        42
         ORG       0.45      0.55      0.49        31
         PER       0.32      0.38      0.35        26

   micro avg       0.47      0.57      0.51        99
   macro avg       0.45      0.54      0.49        99
weighted avg       0.47      0.57      0.51        99

